In [ ]:
%matplotlib inline

import sympy as sp
sp.init_printing(use_latex ='mathjax')
import scipy as sc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import lmfit as lf
import os
import seaborn as sns
from myheatmap import myheatmap
from lmfit import Model
import lmfit
from tabulate import tabulate
import matplotlib

sns.set_context('poster') # makes text larger (poster) or smaller (paper)

In [ ]:
#Define all variables

#individual springs that correspond to individual masses
k1 = sp.symbols('k_1', real = True)
k2 = sp.symbols('k_2', real = True)

#springs that connect two masses
k12 = sp.symbols('k_12', real = True)

#damping coefficients
g1 = sp.symbols('g', real = True)
g2 = sp.symbols('g2', real = True)
 
#masses
m1 = sp.symbols('m1', real = True)
m2 = sp.symbols('m2', real = True)

#Driving force amplitude
F = sp.symbols('F', real = True)

#driving frequency (leave as variable)
wd = sp.symbols('\omega_d', real = True)

In [ ]:
def syserror(x_found,x_set):
    return abs(x_found-x_set)/x_set

In [ ]:
#Solve for driving amplitudes and phase

#Matrix for complex equations of motion
driven = sp.Matrix([[-wd**2*m1 + 1j*wd*g1 + k1 + k12, -k12], [-k12, -wd**2*m2 + 
  1j*wd*g2 + k2 + k12]])

#Matrices for Cramer's Rule
driven_m1 = sp.Matrix([[F, -k12], [0, -wd**2*m2 + 1j*wd*g2 + k2 + k12]])

driven_m2 = sp.Matrix([[-wd**2*m1 + 1j*wd*g1 + k1 + k12, F], [-k12, 0]])

#Apply Cramer's Rule
Cr1, Cr2 = (driven_m1.det()/driven.det(), driven_m2.det()/driven.det())

#Solve for phases for each mass
delta1 = sp.arg(Cr1)
delta2 = sp.arg(Cr2)

#Wrap phases for plots

wrap1 = (delta1)%(2*np.pi)
wrap2 = (delta2)%(2*np.pi)

#Solve for amplitude coefficients
amp1 = sp.Abs(Cr1)
amp2 = sp.Abs(Cr2)


#lambdify curves

c1 = sp.lambdify((wd, k1, k2, k12, g1, g2, F, m1, m2), amp1)
t1 = sp.lambdify((wd, k1, k2, k12, g1, g2, F,  m1, m2), wrap1)

c2 = sp.lambdify((wd, k1, k2, k12, g1, g2, F, m1, m2), amp2)
t2 = sp.lambdify((wd, k1, k2, k12, g1, g2, F, m1, m2), wrap2)

#define functions

#curve = amplitude, theta = phase, e = error (i.e. noise)
def curve1(w, k_1, k_2, k_12, g1_, g2_, F_, m_1, m_2, e):
     return c1(w, k_1, k_2, k_12, g1_, g2_, F_, m_1, m_2) + e
    
def theta1(w, k_1, k_2, k_12, g1_, g2_, F_, m_1, m_2, e):
     return t1(w, k_1, k_2, k_12, g1_, g2_, F_, m_1, m_2) - 2*np.pi + e
    
def curve2(w, k_1, k_2, k_12, g1_, g2_, F_, m_1, m_2, e):
     return c2(w, k_1, k_2, k_12, g1_, g2_, F_, m_1, m_2) + e
    
def theta2(w, k_1, k_2, k_12, g1_, g2_, F_, m_1, m_2, e):
     return t2(w, k_1, k_2, k_12, g1_, g2_, F_, m_1, m_2) - 2*np.pi + e

In [ ]:
#Use functions to make matrices of amplitude and phase for each resonator (with noise)

#define set values
k1_set = 12
k2_set = 27
k12_set = 1
g1_set = 1
g2_set = 0.5
F_set = 1
m1_set = 5
m2_set = 3

vals_set = [m1_set, m2_set, g1_set, g2_set, k1_set, k2_set, k12_set, F_set]

#define number of samples (i.e. range over which to vary the frequency)
n = 50

noiselevel = 1

#define noise (randn(n,) gives a array of normally-distributed random numbers of size n)
amp1_noise = noiselevel* 0.005 * np.random.randn(n,)
phase1_noise = noiselevel* 0.1 * np.random.randn(n,)
amp2_noise = noiselevel* 0.0005 * np.random.randn(n,)
phase2_noise = noiselevel* 0.2 * np.random.randn(n,)

#define driving frequency range (gives array of n evenly spaced numbers between 0.1 and 5)
drive = np.linspace(0.1, 5, num = n)

R1_amp_noiseless = curve1(drive, k1_set, k2_set, k12_set, g1_set, g2_set, F_set, m1_set, m2_set, 0)
R1_phase_noiseless = theta1(drive, k1_set, k2_set, k12_set, g1_set, g2_set, F_set, m1_set, m2_set, 0)
R2_amp_noiseless = curve2(drive, k1_set, k2_set, k12_set, g1_set, g2_set, F_set, m1_set, m2_set, 0)
R2_phase_noiseless = theta2(drive, k1_set, k2_set, k12_set, g1_set, g2_set, F_set, m1_set, m2_set, 0)

usenoise = True
if usenoise: # add a random vector of positive and negative numbers to the curve.
    R1_amp = curve1(drive, k1_set, k2_set, k12_set, g1_set, g2_set, F_set, m1_set, m2_set, amp1_noise)
    R1_phase = theta1(drive, k1_set, k2_set, k12_set, g1_set, g2_set, F_set, m1_set, m2_set, phase1_noise)
    R2_amp = curve2(drive, k1_set, k2_set, k12_set, g1_set, g2_set, F_set, m1_set, m2_set, amp2_noise)
    R2_phase = theta2(drive, k1_set, k2_set, k12_set, g1_set, g2_set, F_set, m1_set, m2_set, phase2_noise)
else:
    R1_amp = R1_amp_noiseless
    R1_phase = R1_phase_noiseless
    R2_amp = R2_amp_noiseless
    R2_phase = R2_phase_noiseless

In [ ]:
amp1_noise

In [ ]:
curve1(1.8, k1_set, k2_set, k12_set, g1_set, g2_set, F_set, m1_set, m2_set, noiselevel* 0.005 * np.random.randn(n,))


In [ ]:
#Choose points for SVD

p = [int(50/4), int(115/4)] #indices of frequencies to use



table = []

for i in range(len(p)):
    table.append([drive[p[i]], R1_amp[p[i]], R1_phase[p[i]], R2_amp[p[i]], R2_phase[p[i]], 
                 syserror(R1_amp[p[i]], R1_amp_noiseless[p[i]]),
                 (R1_phase[p[i]] - R1_phase_noiseless[p[i]]),
                 syserror(R2_amp[p[i]],R2_amp_noiseless[p[i]]),
                 (R2_phase[p[i]]-R2_phase_noiseless[p[i]])])
                 
    
df = pd.DataFrame(data = table, columns = ['drive', 'R1Amp', 'R1Phase', 'R2Amp', 'R2Phase',
                                           'R1Amp_syserror', 'R1Phase_diff', 'R2Amp_syserror', 'R2Phase_diff'])

df

In [ ]:
#Plots of simulated spectra

print('Simulated spectra data, as if from experiment')
print('Resonant frequencies expected at ' 
      + str(np.sqrt(k1_set/m1_set)) + ' and ' 
      + str(np.sqrt(k2_set/m2_set)))

sns.set_context('poster')

fig, ((ax1, ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize = (14,10))

ax1.plot(drive, R1_amp, '.', color = 'silver')
ax1.set_xlabel('Freq (rad/s)')
ax1.set_ylabel('Amplitude\n')
ax1.set_title('Simulated R1 Amp')

#For loop to plot R1 amplitude values from table
for i in range(df.shape[0]):
    ax1.plot(df.drive[i], df.R1Amp[i], 'mo', fillstyle='none', markeredgewidth = 3)
        
ax2.plot(drive, R1_phase, '.', color = 'silver')
ax2.set_xlabel('Freq (rad/s)')
ax2.set_ylabel('Phase (rad)')
ax2.set_title('Simulated R1 Phase')

#For loop to plot R1 amplitude values from table
for i in range(df.shape[0]):
    ax2.plot(df.drive[i], df.R1Phase[i], 'mo', fillstyle='none', markeredgewidth = 3)
        

ax3.plot(drive, R2_amp, '.', color = 'lightsteelblue')
ax3.set_xlabel('Freq (rad/s)')
ax3.set_ylabel('Amplitude\n')
ax3.set_title('Simulated R2 Amp')

#For loop to plot R1 amplitude values from table
for i in range(df.shape[0]):
    ax3.plot(df.drive[i], df.R2Amp[i], 'mo', fillstyle='none', markeredgewidth = 3)
    

ax4.plot(drive, R2_phase, '.', color = 'lightsteelblue')
ax4.set_xlabel('Freq (rad/s)')
ax4.set_ylabel('Phase (rad)')
ax4.set_title('Simulated R2 Phase')

#For loop to plot R1 amplitude values from table
for i in range(df.shape[0]):
    ax4.plot(df.drive[i], df.R2Phase[i], 'mo', fillstyle='none', markeredgewidth = 3)
        

plt.tight_layout()

In [ ]:
#Define function to calculate phase correction and complex amplitudes

print('Simulated response vector data, as if from experiment')

ampComplex = []

def complexAmp(df):
    
    #Use amplitude and phase to calculate complex amplitude
    Z1 = df['R1Amp'] * np.exp(1j * df['R1Phase'])
    Z2 = df['R2Amp'] * np.exp(1j * df['R2Phase'])
    
    #Add complex amplitude to table to append to dataframe
    ampComplex.append([Z1, Z2])

for i in range(df.shape[0]):
    complexAmp(df.iloc[i])

df['R1AmpCom'], df['R2AmpCom'] = np.transpose(ampComplex)

df

In [ ]:
#Singular Value Decomposition

verbose = False

#define which two points to use for analysis
vals1 = df.loc[0]
vals2 = df.loc[1]
if verbose:
    display("vals1:")
    display(vals1)
    display("vals2:")
    display(vals2)

#define values for svd
d1 = vals1['drive']
d1Z1 = vals1['R1AmpCom']
d1Z2 = vals1['R2AmpCom']

d2 = vals2['drive']
d2Z1 = vals2['R1AmpCom']
d2Z2 = vals2['R2AmpCom']

#Define Matrix M
M = np.array([[-d1**2*np.real(d1Z1), 0, -d1*np.imag(d1Z1), 0, np.real(d1Z1), 0, np.real(d1Z1)-np.real(d1Z2), -1],
        [-d1**2*np.imag(d1Z1), 0, d1*np.real(d1Z1), 0, np.imag(d1Z1), 0, np.imag(d1Z1)-np.imag(d1Z2), 0],
        [0, -d1**2*np.real(d1Z2), 0, -d1*np.imag(d1Z2), 0, np.real(d1Z2), np.real(d1Z2)-np.real(d1Z1), 0],
        [0, -d1**2*np.imag(d1Z2), 0, d1*np.real(d1Z2), 0, np.imag(d1Z2), np.imag(d1Z2)-np.imag(d1Z1), 0],
        [-d2**2*np.real(d2Z1), 0, -d2*np.imag(d2Z1), 0, np.real(d2Z1), 0, np.real(d2Z1)-np.real(d2Z2), -1],
        [-d2**2*np.imag(d2Z1), 0, d2*np.real(d2Z1), 0, np.imag(d2Z1), 0, np.imag(d2Z1)-np.imag(d2Z2), 0],
        [0, -d2**2*np.real(d2Z2), 0, -d2*np.imag(d2Z2), 0, np.real(d2Z2), np.real(d2Z2)-np.real(d2Z1), 0],
        [0, -d2**2*np.imag(d2Z2), 0, d2*np.real(d2Z2), 0, np.imag(d2Z2), np.imag(d2Z2)-np.imag(d2Z1), 0]])

if verbose:
    display("M matrix:")
    display(M)

#SVD
u, s, vh = np.linalg.svd(M, full_matrices = True)
#u, s, vh = sc.linalg.svd(M, full_matrices = False, lapack_driver = 'gesvd')

# the smallest singular value is always the last one, index -1
print(s)

plt.semilogy(s, '.')
plt.title('singular values')

In [ ]:
# singular vectors. # M1, M2, G1, G2, K1, K2, K12, FD
vh

In [ ]:
print("Checking the python-normalization")

vect =vh[-1]
sq = [el**2 for el in vect]
sum(sq)

In [ ]:
[M1, M2, G1, G2, K1, K2, K12, FD] = vh[-1]

vals = [['Norm m1', 'Norm m2', 'Norm g1', 'Norm g2', 'Norm k1', 'Norm k2','Norm c12', 'Norm Driving Force']]
vals.append((M1, M2, G1, G2, K1, K2, K12, FD))

print("Python-Normalized singular vector corresponding to elements vector")
print(tabulate(vals, headers = 'firstrow', tablefmt = 'fancy_grid'))

In [ ]:
#View vh array and assign variables to proper row vector

print('Results of analysis versus set values:')
print("The python normalization loses one degree of freedom, \
so we must be able to independently determine one element; say it's the driving force")

c = F_set / FD
elements = (M1 * c, M2 * c, G1 * c, G2 * c, K1 * c, K2 * c, K12 * c, FD * c)
syserrors = [syserror(elements[i], vals_set[i]) for i in range(len(elements))]

vals2 = [['m1', 'm2', 'g1', 'g2', 'k1', 'k2','c12', 'Driving Force']]
vals2.append(elements)
vals2.append(vals_set)
vals2.append(syserrors)

print(tabulate(vals2, headers = 'firstrow', tablefmt = 'fancy_grid'))

In [ ]:
elementsdf = pd.DataFrame(vals2, index = ['labels', 'from_svd', 'set', "syserror"])
new_header = elementsdf.iloc[0] 
elementsdf.columns = new_header

display(elementsdf)

print('Noise level: ' + str(noiselevel))
(elementsdf.transpose()['syserror']*100).plot(kind='bar')
plt.gca().set_ylabel('systematic error (%)\n')
plt.gca().set_xlabel('');


In [ ]:
## what if the null-space is 2D?

#View vh array and assign variables to proper row vector

print('Results of analysis versus set values:')
print("If the null-space is 2D, we must be able to independently determine two elements; say it's m1 and m2")
## but it would be cool if it were k1/m1 and k2/m2 because that should be known from the measured resonant frequencies

# elements vector: 'm1', 'm2', 'g1', 'g2', 'k1', 'k2','c12', 'Driving Force'
vect1 = vh[-1]
vect2 = vh[-2]

# find linear combination such that:
# a * vect1[0] + b * vect2[0] = m1_set   and
# a * vect1[1] + b * vect2[1] = m2_set
## But this rearranges to:

coefa = ( vect2[1] * m1_set - m2_set * vect2[0] ) / (vect2[1]*vect1[0] - vect1[1]*vect2[0] )
coefb = (vect1[1]*m1_set - m2_set *vect1[0] ) /(vect1[1]*vect2[0] - vect2[1]*vect1[0] )

elements = [coefa*vect1[k]+coefb*vect2[k]  for k in range(len(vect1)) ]
syserrors = [syserror(elements[i], vals_set[i]) for i in range(len(elements))]

vals3 = [['m1', 'm2', 'g1', 'g2', 'k1', 'k2','c12', 'Driving Force']]
vals3.append(elements)
vals3.append(vals_set)
vals3.append(syserrors)

print(tabulate(vals3, headers = 'firstrow', tablefmt = 'fancy_grid'))

elementsdf3 = pd.DataFrame(vals3, index = ['labels', 'from_svd', 'set', "syserror"])
new_header3 = elementsdf3.iloc[0] 
elementsdf3.columns = new_header3

display(elementsdf3)

print('Noise level: ' + str(noiselevel))
(elementsdf3.transpose()['syserror']*100).plot(kind='bar')
plt.gca().set_ylabel('systematic error (%)\n')
plt.gca().set_xlabel('');


In [ ]:
#Plots of singular value decomposition

sns.set_context('poster')

fig, ((ax1, ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize = (14,10))

ax1.plot(drive, R1_amp, '.', color = 'silver')
ax1.plot(drive, curve1(drive, K1, K2, K12, G1, G2, FD, M1, M2, 0), '--', color='black')
ax1.set_xlabel('Freq (rad/s)')
ax1.set_ylabel('Amplitude\n')
ax1.set_title('Simulated R1 Amp')

#For loop to plot R1 amplitude values from table
for i in range(df.shape[0]):
    ax1.plot(df.drive[i], df.R1Amp[i], 'mo', fillstyle='none', markeredgewidth = 3)
        
ax2.plot(drive, R1_phase, '.', color = 'silver')
ax2.plot(drive, theta1(drive, K1, K2, K12, G1, G2, FD, M1, M2, 0), '--', color='black')
ax2.set_xlabel('Freq (rad/s)')
ax2.set_ylabel('Phase (rad)')
ax2.set_title('Simulated R1 Phase')

#For loop to plot R1 amplitude values from table
for i in range(df.shape[0]):
    ax2.plot(df.drive[i], df.R1Phase[i], 'mo', fillstyle='none', markeredgewidth = 3)
        

ax3.plot(drive, R2_amp, '.', color = 'lightsteelblue')
ax3.plot(drive, curve2(drive, K1, K2, K12, G1, G2, FD, M1, M2, 0), '--', color='black')
ax3.set_xlabel('Freq (rad/s)')
ax3.set_ylabel('Amplitude\n')
ax3.set_title('Simulated R2 Amp')

#For loop to plot R1 amplitude values from table
for i in range(df.shape[0]):
    ax3.plot(df.drive[i], df.R2Amp[i], 'mo', fillstyle='none', markeredgewidth = 3)
    

ax4.plot(drive, R2_phase, '.', color = 'lightsteelblue')
ax4.plot(drive, theta2(drive, K1, K2, K12, G1, G2, FD, M1, M2, 0), '--', color='black')
ax4.set_xlabel('Freq (rad/s)')
ax4.set_ylabel('Phase (rad)')
ax4.set_title('Simulated R2 Phase')

#For loop to plot R1 amplitude values from table
for i in range(df.shape[0]):
    ax4.plot(df.drive[i], df.R2Phase[i], 'mo', fillstyle='none', markeredgewidth = 3)
        

plt.tight_layout()

In [ ]:
#Calculate "badness of fit" parameter

amp1_badness = np.sum((R1_amp - curve1(drive, K1, K2, K12, G1, G2, FD, M1, M2, 0))**2)
phase1_badness = np.sum((R1_phase - theta1(drive, K1, K2, K12, G1, G2, FD, M1, M2, 0))**2)
amp2_badness = np.sum((R2_amp - curve2(drive, K1, K2, K12, G1, G2, FD, M1, M2, 0))**2)
phase2_badness = np.sum((R2_phase - theta2(drive, K1, K2, K12, G1, G2, FD, M1, M2, 0))**2)

print("R1 Amp Badness of Fit = ", amp1_badness,
     "\nR1 Phase Badness of Fit = ", phase1_badness,
      "\nR2 Amp Badness of Fit = ", amp2_badness,
      "\nR2 Phase Badness of Fit = ", phase2_badness)

**Sweep through possible point differences**

In [ ]:
#Code that loops through points of different spacing

#n = 30

#initiate arrays for badness of fit values
fits = []

# initate array for systematic error sum of squares values
results = []

#Start table, to be indexed by frequency indices
tab = []

# Loop over possible combinations of frequency indices, i1 and i2
for i1 in range(n):
    
    # Values for frequency 1
    freq1_data = [[drive[i1], R1_amp[i1], R1_phase[i1], R2_amp[i1], R2_phase[i1]]]
    
    freq1_df_vals = pd.DataFrame(data = freq1_data, columns = ['drive', 'R1Amp', 'R1Phase', 'R2Amp', 'R2Phase'])
    
    for i2 in range(n):
        # Values for frequency 2
        freq2_data = [[drive[i2], R1_amp[i2], R1_phase[i2], R2_amp[i2], R2_phase[i2]]]

        freq2_df_vals = pd.DataFrame(data = freq2_data, columns = ['drive', 'R1Amp', 'R1Phase', 'R2Amp', 'R2Phase'])

        #Calculate complex values   
        ampComplex = []

        #Complex amplitudes of both resonators with frequency 1
        Z1_1 = R1_amp[i1] * np.exp(1j * R1_phase[i1])
        Z2_1 = R2_amp[i1] * np.exp(1j * R2_phase[i1])
        ampComplex.append([Z1_1, Z2_1])

        #Complex amplitudes of both resonators with frequency 2
        Z1_2 = R1_amp[i2] * np.exp(1j * R1_phase[i2])
        Z2_2 = R2_amp[i2] * np.exp(1j * R2_phase[i2])
        ampComplex.append([Z1_2, Z2_2])

        freq1_df_vals['R1AmpCom'] = Z1_1
        freq1_df_vals['R2AmpCom'] = Z2_1
        freq2_df_vals['R1AmpCom'] = Z1_2
        freq2_df_vals['R2AmpCom'] = Z2_2

        #define which two points to use for analysis
        vals1 = freq1_df_vals.loc[0]
        vals2 = freq2_df_vals.loc[0]

        #define values for svd
        d1 = vals1['drive']
        d1Z1 = vals1['R1AmpCom']
        d1Z2 = vals1['R2AmpCom']

        d2 = vals2['drive']
        d2Z1 = vals2['R1AmpCom']
        d2Z2 = vals2['R2AmpCom']

        M = np.array([[-d1**2*np.real(d1Z1), 0, -d1*np.imag(d1Z1), 0, np.real(d1Z1), 0, np.real(d1Z1)-np.real(d1Z2), -1],
            [-d1**2*np.imag(d1Z1), 0, d1*np.real(d1Z1), 0, np.imag(d1Z1), 0, np.imag(d1Z1)-np.imag(d1Z2), 0],
            [0, -d1**2*np.real(d1Z2), 0, -d1*np.imag(d1Z2), 0, np.real(d1Z2), np.real(d1Z2)-np.real(d1Z1), 0],
            [0, -d1**2*np.imag(d1Z2), 0, d1*np.real(d1Z2), 0, np.imag(d1Z2), np.imag(d1Z2)-np.imag(d1Z1), 0],
            [-d2**2*np.real(d2Z1), 0, -d2*np.imag(d2Z1), 0, np.real(d2Z1), 0, np.real(d2Z1)-np.real(d2Z2), -1],
            [-d2**2*np.imag(d2Z1), 0, d2*np.real(d2Z1), 0, np.imag(d2Z1), 0, np.imag(d2Z1)-np.imag(d2Z2), 0],
            [0, -d2**2*np.real(d2Z2), 0, -d2*np.imag(d2Z2), 0, np.real(d2Z2), np.real(d2Z2)-np.real(d2Z1), 0],
            [0, -d2**2*np.imag(d2Z2), 0, d2*np.real(d2Z2), 0, np.imag(d2Z2), np.imag(d2Z2)-np.imag(d2Z1), 0]])

        #SVD
        u, s, vh = np.linalg.svd(M, full_matrices = True)

        # extract parameters found by SVD
        #assign variables # mass of resonator 1, mass 2, damping 1, damping 2, stiffness 1, stiffness 2, coupling stiffness, force
        [M1, M2, G1, G2, K1, K2, K12, FD] = vh[-1] # the 7th singular value is the smallest one (closest to zero)

        # too add: normalize elements vector ***
        
        # Alternative check: 
        # calculate how close the SVD-determined parameters are compared to the originally set parameters

        # Values to compare:
        # Set values: k1_set, k2_set, k12_set, g1_set, g2_set, F_set, m1_set, m2_set
        # SVD-determined values: M1, M2, G1, G2, K1, K2, K12, FD
        K1syserror = syserror(K1,k1_set)
        K2syserror = syserror(K2,k2_set)
        K12syserror = syserror(K12,k12_set)
        G1syserror = syserror(G1,g1_set)
        G2syserror = syserror(G2,g2_set)
        Fsyserror = syserror(FD,F_set)
        m1syserror = syserror(M1,m1_set)
        m2syserror = syserror(M2,m2_set)
        parametererror = np.sqrt((K1syserror**2 + K2syserror**2 + K12syserror**2 + G1syserror**2 + G2syserror**2 + Fsyserror**2 + \
                        m1syserror**2 + m2syserror**2)/8)
        results.append([drive[i1],
                          drive[i2],
                          M1, M2, G1, G2, K1, K2, K12, FD,
                          K1syserror,
                          K2syserror,
                          K12syserror,
                          G1syserror,
                          G2syserror,
                          Fsyserror,
                          m1syserror,
                          m2syserror,
                          parametererror])
        #display(parametererror)


        #calculate badness of fit for R1 and R2 amplitudes and phases and add to arrays
        fits.append([drive[i1], drive[i2], 
                     np.sum(R1_amp - curve1(drive, K1, K2, K12, G1, G2, FD, M1, M2, 0))**2, 
                    np.sum((R1_phase - theta1(drive, K1, K2, K12, G1, G2, FD, M1, M2, 0))**2),
                    np.sum((R2_amp - curve2(drive, K1, K2, K12, G1, G2, FD, M1, M2, 0))**2),
                    np.sum((R2_phase - theta2(drive, K1, K2, K12, G1, G2, FD, M1, M2, 0))**2)])
        
diffs = pd.DataFrame(data = fits, columns = ['Freq1', 'Freq2', 'R1AmpFit', 'R1PhaseFit', 'R2AmpFit', 'R2PhaseFit'])

resultsdf = pd.DataFrame(data=results, columns = ['Freq1', 
                                                    'Freq2',
                                                    'M1', 'M2', 'G1', 'G2', 'K1', 'K2', 'K12', 'FD',
                                                    'K1syserror',
                                                    'K2syserror',
                                                    'K12syserror',
                                                    'G1syserror',
                                                    'G2syserror',
                                                    'Fsyserror',
                                                    'm1syserror',
                                                    'm2syserror',
                                                    'parametererror'])

In [ ]:
resultsdf.head()

In [ ]:
print('How much variation is there in the parameters that obtained from the frequency sweep?')

# Take absolute value because the null space vector could be normalized as positive or negative
temp = np.double(np.array(abs(resultsdf.M1)))
#display(temp)
display(min(temp))
display(max(temp))
#plt.plot(temp, '.')

max(temp) - min(temp)

In [ ]:
errgrid=resultsdf.pivot_table(index = 'Freq1', columns = 'Freq2', values = 'm2syserror').sort_index(axis = 0, ascending = False)
axx, cbar1 = myheatmap(errgrid, "m2 sys error", return_cbar=True); 
axx.axis('equal');

In [ ]:
SSgrid=resultsdf.pivot_table(index = 'Freq1', columns = 'Freq2', values = 'parametererror').sort_index(axis = 0, ascending = False)
axx, cbar1 = myheatmap(SSgrid, "SS sys error", return_cbar=True); 
axx.axis('equal');
print("Why isn't this symmetric? Is SVD sensitive to which frequency comes first?")

In [ ]:
## Plot systematic error of parameters, sum of squares, as a function of the higher freq ***
sns.set_context('poster')

fig, ((ax1, ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize = (14,10))

ax1.plot(resultsdf.Freq2,resultsdf.parametererror, '.')
ax1.set_xlabel('Frequency 2 (rad/s)')
ax1.set_ylabel('Sys error SS')
#ax1.set_title('Systematic error sum of squares')



ax2.plot(resultsdf.Freq2,abs(resultsdf.M1), '.')
ax2.set_ylabel('absolute value of Mass 1')
ax2.set_xlabel('Frequency 2 (rad/s)')

ax3.plot(resultsdf.Freq2,(resultsdf.M1), '.')
ax3.set_ylabel('Mass 1')
ax3.set_xlabel('Frequency 2 (rad/s)')

ax4.plot(resultsdf.Freq2,abs(resultsdf.G2), '.')
ax4.set_ylabel('absolute value of Gamma 2')
ax4.set_xlabel('Frequency 2 (rad/s)')

plt.tight_layout()


In [ ]:
"""#Plots of badness of fits as a function of seperation between points
sns.set_context('poster')

fig, ((ax1, ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize = (14,10))

ax1.plot(diffs.Difference, (diffs.R1AmpFit), '.', color = 'silver')
ax1.set_xlabel('Frequency Difference (rad/s)')
ax1.set_ylabel('Difference Squared ($m^2$)')
ax1.set_title('R1 Amp Badness of Fit')
        
ax2.plot(diffs.Difference, (diffs.R1PhaseFit), '.', color = 'silver')
ax2.set_xlabel('Frequency Difference (rad/s)')
ax2.set_ylabel('Difference Squared ($rad^2$)')
ax2.set_title('R1 Phase Badness of Fit')     

ax3.plot(diffs.Difference, (diffs.R2AmpFit), '.', color = 'lightsteelblue')
ax3.set_xlabel('Frequency Difference (rad/s)')
ax3.set_ylabel('Difference Squared ($m^2$)')
ax3.set_title('R2 Amp Badness of Fit')
    
ax4.plot(diffs.Difference, (diffs.R2PhaseFit), '.', color = 'lightsteelblue')
ax4.set_xlabel('Frequency Difference (rad/s)')
ax4.set_ylabel('Difference Squared ($rad^2$)')
ax4.set_title('R2 Phase Badness of Fit')
        
plt.tight_layout()

#Print frequency difference with smallest badness of fit

print("R1 Amp min = ", diffs.iloc[diffs.R1AmpFit.idxmin()].Difference, "m",
     "\nR1 Phase min = ", diffs.iloc[diffs.R1PhaseFit.idxmin()].Difference, "rad",
     "\nR2 Amp min = ", diffs.iloc[diffs.R2AmpFit.idxmin()].Difference, "m",
     "\nR2 Phase min = ", diffs.iloc[diffs.R2PhaseFit.idxmin()].Difference, "rad")"""

In [ ]:
#Plot combined amplitude and phase Badness of Fit parameters

#Plots of badness of fits as a function of seperation between points
sns.set_context('poster')

fig, ((ax1, ax2)) = plt.subplots(1, 2, figsize = (14,5))

ax1.plot(diffs.Difference, (diffs.R1AmpFit)+(diffs.R2AmpFit), '.', color = 'silver')
ax1.set_xlabel('Frequency Difference (rad/s)')
ax1.set_ylabel('Difference Squared ($m^2$)')
ax1.set_title('Amplitude Badness of Fit')
        
ax2.plot(diffs.Difference, (diffs.R1PhaseFit)+(diffs.R2PhaseFit), '.', color = 'silver')
ax2.set_xlabel('Frequency Difference (rad/s)')
ax2.set_ylabel('Difference Squared ($rad^2$)')
ax2.set_title('Phase Badness of Fit')     

plt.tight_layout()

#Print frequency difference with smallest badness of fit

print("Amp min = ", diffs.iloc[diffs.R1AmpFit.idxmin()].Difference + 
      diffs.iloc[diffs.R2AmpFit.idxmin()].Difference, "m",
     "\nR1 Phase min = ", diffs.iloc[diffs.R1PhaseFit.idxmin()].Difference +
      diffs.iloc[diffs.R2PhaseFit.idxmin()].Difference, "rad")

In [ ]:
#Plots of badness of fits as a function of seperation between points
sns.set_context('poster')

fig, (ax1) = plt.subplots(1, 1, figsize = (8,6))

ax1.plot(diffs.Difference, np.log(diffs.R1AmpFit + diffs.R2AmpFit + diffs.R1PhaseFit + diffs.R2PhaseFit), '.', color = 'silver')
ax1.set_xlabel('Frequency Difference (rad/s)')
ax1.set_ylabel('Log Difference Squared ($m^2$)')
ax1.set_title('Total Badness of Fit')

print("Total min = ", diffs.iloc[(diffs.R1AmpFit + diffs.R2AmpFit + diffs.R1PhaseFit + diffs.R2PhaseFit).idxmin()].Difference, "rad/s")

**Sweep through starting points**

In [ ]:
#Code that loops through points of different spacing

interval = 10

j = 1

max_vals = []

while j < (n - interval):
    
    #Define starting point and interval
    start = j


    #initiate arrays for badness of fit values
    fits = []

    #Start table with starting point
    tab = [[drive[start], R1_amp[start], R1_phase[start], R2_amp[start], R2_phase[start]]]

    i = 1 

    while i < (n - start):
    
        #Append next data point to table and add to dataframe
        tab.append([drive[start + i], R1_amp[start + i], R1_phase[start + i], R2_amp[start + i], R2_phase[start + i]])
    
        df_vals = pd.DataFrame(data = tab, columns = ['drive', 'R1Amp', 'R1Phase', 'R2Amp', 'R2Phase'])
    
        #Calculate complex values   
        ampComplex = []
    
        Z1_1 = df_vals['R1Amp'][0] * np.exp(1j * df_vals['R1Phase'][0])
        Z2_1 = df_vals['R2Amp'][0] * np.exp(1j * df_vals['R2Phase'][0])
    
        Z1_2 = df_vals['R1Amp'][1] * np.exp(1j * df_vals['R1Phase'][1])
        Z2_2 = df_vals['R2Amp'][1] * np.exp(1j * df_vals['R2Phase'][1])
    
        ampComplex.append([Z1_1, Z2_1])
        ampComplex.append([Z1_2, Z2_2])

        df_vals['R1AmpCom'], df_vals['R2AmpCom'] = np.transpose(ampComplex)

        #define which two points to use for analysis
        vals1 = df_vals.loc[0]
        vals2 = df_vals.loc[1]

        #define values for svd
        d1 = vals1['drive']
        d1Z1 = vals1['R1AmpCom']
        d1Z2 = vals1['R2AmpCom']

        d2 = vals2['drive']
        d2Z1 = vals2['R1AmpCom']
        d2Z2 = vals2['R2AmpCom']
    
        M = np.array([[-d1**2*np.real(d1Z1), 0, -d1*np.imag(d1Z1), 0, np.real(d1Z1), 0, np.real(d1Z1)-np.real(d1Z2), -1],
            [-d1**2*np.imag(d1Z1), 0, d1*np.real(d1Z1), 0, np.imag(d1Z1), 0, np.imag(d1Z1)-np.imag(d1Z2), 0],
            [0, -d1**2*np.real(d1Z2), 0, -d1*np.imag(d1Z2), 0, np.real(d1Z2), np.real(d1Z2)-np.real(d1Z1), 0],
            [0, -d1**2*np.imag(d1Z2), 0, d1*np.real(d1Z2), 0, np.imag(d1Z2), np.imag(d1Z2)-np.imag(d1Z1), 0],
            [-d2**2*np.real(d2Z1), 0, -d2*np.imag(d2Z1), 0, np.real(d2Z1), 0, np.real(d2Z1)-np.real(d2Z2), -1],
            [-d2**2*np.imag(d2Z1), 0, d2*np.real(d2Z1), 0, np.imag(d2Z1), 0, np.imag(d2Z1)-np.imag(d2Z2), 0],
            [0, -d2**2*np.real(d2Z2), 0, -d2*np.imag(d2Z2), 0, np.real(d2Z2), np.real(d2Z2)-np.real(d2Z1), 0],
            [0, -d2**2*np.imag(d2Z2), 0, d2*np.real(d2Z2), 0, np.imag(d2Z2), np.imag(d2Z2)-np.imag(d2Z1), 0]])

        #SVD
        u, s, vh = np.linalg.svd(M, full_matrices = True)
    
        #assign variables
        [M1, M2, G1, G2, K1, K2, K12, FD] = vh[7]
    
        #calculate badness of fit for R1 and R2 amplitudes and phases and add to arrays
        max_vals.append([drive[start], drive[start + i] - drive[start], 
                np.sum(R1_amp - curve1(drive, K1, K2, K12, G1, G2, FD, M1, M2, 0))**2 + 
                np.sum((R1_phase - theta1(drive, K1, K2, K12, G1, G2, FD, M1, M2, 0))**2) +
                np.sum((R2_amp - curve2(drive, K1, K2, K12, G1, G2, FD, M1, M2, 0))**2) +
                np.sum((R2_phase - theta2(drive, K1, K2, K12, G1, G2, FD, M1, M2, 0))**2)])
    
        #remove data from table and dataframe to start over
        tab.pop()
    
        i = i + interval
        
    j = j + interval
    
maxes = pd.DataFrame(data = max_vals, columns = ['Starting Freq', 'FreqDiff', 'TotalFit'])

In [ ]:
#maxes

In [ ]:
#Plots of badness of fits as a function of seperation between points
sns.set_context('poster')

fig, (ax1) = plt.subplots(1, 1, figsize = (8,6))

ax1.plot(maxes.FreqDiff, (maxes.TotalFit), '.', color = 'silver')
ax1.set_xlabel('Frequency Difference (rad/s)')
ax1.set_ylabel('Difference Squared ($m^2$)')
ax1.set_title('Total Badness of Fit')

# Sweep error to see effect of difference squared values

In [ ]:
#define points for SVD

points = [50, 70]

#define drive frequencies
drive = np.linspace(0.1, 5, num = n)

results = []

amplitudeadder = 0

interval = 0.001

#Amplitudes and phases with no error
R1amp = curve1(drive, k1_set, k2_set, k12_set, g1_set, g2_set, F_set, m1_set, m2_set, 0)
R1phase = theta1(drive, k1_set, k2_set, k12_set, g1_set, g2_set, F_set, m1_set, m2_set, 0)
R2amp = curve2(drive, k1_set, k2_set, k12_set, g1_set, g2_set, F_set, m1_set, m2_set, 0)
R2phase = theta2(drive, k1_set, k2_set, k12_set, g1_set, g2_set, F_set, m1_set, m2_set, 0)

def complexAmp(df, ampComplex=[]):
    #Use amplitude and phase to calculate complex amplitude
    Z1 = df['R1Amp'] * np.exp(1j * df['R1Phase'])
    Z2 = df['R2Amp'] * np.exp(1j * df['R2Phase'])

    #Add complex amplitude to table to append to dataframe
    ampComplex.append([Z1, Z2])

while amplitudeadder < 0.05 :

    #Put amplitude and phase values in table and add error
    tabl = []
    
    for j in range(len(points)):
        tabl.append([drive[points[j]], R1amp[points[j]] + amplitudeadder, R1amp[points[j]] + amplitudeadder,
                      R2amp[points[j]] + amplitudeadder, R2amp[points[j]] + amplitudeadder])

    df = pd.DataFrame(data = tabl, columns = ['drive', 'R1Amp', 'R1Phase', 'R2Amp', 'R2Phase'])

    #Define complex amplitudes
    #Define function to calculate phase correction and complex amplitudes

    ampComplex = []

    for j in range(df.shape[0]):
        complexAmp(df.iloc[j], ampComplex)

    df['R1AmpCom'], df['R2AmpCom'] = np.transpose(ampComplex)
    
    #Singular Value Decomposition

    #define which two points to use for analysis
    vals1 = df.loc[0]
    vals2 = df.loc[1]

    #define values for svd
    d1 = vals1['drive']
    d1Z1 = vals1['R1AmpCom']
    d1Z2 = vals1['R2AmpCom']

    d2 = vals2['drive']
    d2Z1 = vals2['R1AmpCom']
    d2Z2 = vals2['R2AmpCom']

    #Define Matrix M
    M = np.array([[-d1**2*np.real(d1Z1), 0, -d1*np.imag(d1Z1), 0, np.real(d1Z1), 0, np.real(d1Z1)-np.real(d1Z2), -1],
            [-d1**2*np.imag(d1Z1), 0, d1*np.real(d1Z1), 0, np.imag(d1Z1), 0, np.imag(d1Z1)-np.imag(d1Z2), 0],
            [0, -d1**2*np.real(d1Z2), 0, -d1*np.imag(d1Z2), 0, np.real(d1Z2), np.real(d1Z2)-np.real(d1Z1), 0],
            [0, -d1**2*np.imag(d1Z2), 0, d1*np.real(d1Z2), 0, np.imag(d1Z2), np.imag(d1Z2)-np.imag(d1Z1), 0],
            [-d2**2*np.real(d2Z1), 0, -d2*np.imag(d2Z1), 0, np.real(d2Z1), 0, np.real(d2Z1)-np.real(d2Z2), -1],
            [-d2**2*np.imag(d2Z1), 0, d2*np.real(d2Z1), 0, np.imag(d2Z1), 0, np.imag(d2Z1)-np.imag(d2Z2), 0],
            [0, -d2**2*np.real(d2Z2), 0, -d2*np.imag(d2Z2), 0, np.real(d2Z2), np.real(d2Z2)-np.real(d2Z1), 0],
            [0, -d2**2*np.imag(d2Z2), 0, d2*np.real(d2Z2), 0, np.imag(d2Z2), np.imag(d2Z2)-np.imag(d2Z1), 0]])

    #SVD
    u, s, vh = np.linalg.svd(M, full_matrices = True)
    #u, s, vh = sc.linalg.svd(M, full_matrices = False, lapack_driver = 'gesvd')

    
    #assign variables
    [M1, M2, G1, G2, K1, K2, K12, FD] = vh[7]
    
    #calculate badness of fit for R1 and R2 amplitudes and phases and add to arrays
    results.append([drive[points[1]] - drive[points[0]], amplitudeadder,
                np.sum(R1amp - curve1(drive, K1, K2, K12, G1, G2, FD, M1, M2, 0))**2, 
                np.sum((R1phase - theta1(drive, K1, K2, K12, G1, G2, FD, M1, M2, 0))**2),
                np.sum((R2amp - curve2(drive, K1, K2, K12, G1, G2, FD, M1, M2, 0))**2),
                np.sum((R2phase - theta2(drive, K1, K2, K12, G1, G2, FD, M1, M2, 0))**2)])
    
    #remove data from table and dataframe to start over
    tabl.pop()
    
    amplitudeadder = amplitudeadder + interval
    
errors = pd.DataFrame(data = results, columns = ['Difference','error', 'R1AmpFit', 'R1PhaseFit', 'R2AmpFit', 'R2PhaseFit'])

In [ ]:
#Plot combined amplitude and phase Badness of Fit parameters

#Plots of badness of fits as a function of seperation between points
sns.set_context('poster')

fig, ((ax1, ax2)) = plt.subplots(1, 2, figsize = (14,5))

ax1.plot(errors.error, (errors.R1AmpFit)+(errors.R2AmpFit), '.', color = 'silver')
ax1.set_xlabel('amplitudeadder')
ax1.set_ylabel('Difference Squared ($m^2$)')
ax1.set_title('Amplitude Badness of Fit')
        
ax2.plot(errors.error, (errors.R1PhaseFit)+(errors.R2PhaseFit), '.', color = 'silver')
ax2.set_xlabel('amplitudeadder')
ax2.set_ylabel('Difference Squared ($rad^2$)')
ax2.set_title('Phase Badness of Fit')     

plt.tight_layout()